In [ ]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession 
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from pyspark.sql import Row,SQLContext
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import requests
import sys
import time
import json

In [ ]:
ss = SparkSession.Builder() \
     .appName("Realtime") \
     .master("spark://spark-master-1:7077") \
     .config("spark.jars", "./spark-streaming-kafka-0-8-assembly_2.11-2.0.0-preview.jar") \
     .getOrCreate()
sc = ss.sparkContext
ssc = StreamingContext(sc, 2)
ssc.checkpoint("checkpoint_TwitterApp")
ss.sparkContext.setLogLevel('WARN')

In [ ]:
brokers = 'kafka-1:9092,kafka-2:9092'
topic='tweet'

In [ ]:
def filter(x):
    if x[0] == '#':
        if len(x) != 1:
            return True
    return False

In [ ]:
# lines = ks.window(30,10)
lines = ks.map(lambda x: json.loads(x[1])['text'] )
words = lines.flatMap(lambda line: line.split())
hashtags = words.filter(lambda w: filter(w)).map(lambda x: (x, 1))
tags_totals = hashtags.reduceByKey(lambda a, b: a+b)\
                .transform(lambda rdd: rdd.sortBy(lambda x: x[1], ascending=False))
tags_totals.foreachRDD(process_rdd)    
ssc.start()
time.sleep(1000)
ssc.awaitTermination()

In [ ]:
def sentimentAnalysis(ssc):
    es = Elasticsearch()
    es_write_conf = { "es.resource" : "tsd/mem","es.input.json" : "true" } 
    kstream = KafkaUtils.createDirectStream(ssc, [topic], kafkaParams={"metadata.broker.list":brokers})
    tweets  = kstream.map(lambda line: json.loads(line[1]))
    sentiAnalysis = tweets.map(lambda line: (line,classify(line["text"])))
    result2 = result1.map(lambda line: (None, line))
    result2.foreachRDD(lambda line : line.saveAsNewAPIHadoopFile(
            path="-", 
            outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat",
            keyClass="org.apache.hadoop.io.NullWritable", 
            valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable", 
            conf=es_write_conf))
	ssc.start()
	ssc.awaitTermination()
	